In [3]:
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
import pathlib


In [2]:
import sklearn

M, m, p = sklearn.__version__.split('.')
display(sklearn.__version__)
assert int(M) == 1
assert int(m) >= 2

AssertionError: 

# Load Data

In [ ]:
data_folders = [
    pathlib.Path(f'./data/bicing/truncated/{y}') for y in [2022]
]

In [ ]:
pandas_kwargs = {
    'index_col': 0,
}
df = pd.concat([pd.read_csv(file, **pandas_kwargs) for data_folder in data_folders for file in
                data_folder.glob('*/*.csv')]).drop_duplicates()

In [ ]:
df = df.assign(
    datetime=lambda x: pd.to_datetime(x.datetime),
    percentage_docks_available=df.num_docks_available / (df.num_docks_available + df.num_bikes_available),
).drop(columns=['num_docks_available', 'num_bikes_available']).sort_values(['station_id', 'datetime'])

In [ ]:
df

### Add station info

In [ ]:
station_info = pd.read_csv('./data/bicing_info.csv')
df = pd.merge(left=df, right=station_info[['station_id', 'lat', 'lon', 'altitude', 'post_code']],
              on=['station_id'])
df.head()

### climate

In [ ]:
df_climate_ = pd.read_csv('./data/clima.csv', parse_dates=['time'])

df_climate = df_climate_.assign(
    year=df_climate_.time.dt.year,
    month=df_climate_.time.dt.month,
    day=df_climate_.time.dt.day,
    hour=df_climate_.time.dt.hour
)
df = pd.merge(left=df, right=df_climate.drop(columns=['time']), on=['hour', 'day', 'month', 'year'])

In [ ]:
df

In [ ]:
import numpy as np

# merge with previous hours
df = df.sort_values(['station_id', 'datetime']).assign(
    day_of_week=lambda x: x.datetime.dt.day_of_week,
    is_weekend=lambda x: x.day_of_week >= 5,
    is_night=lambda x: np.bitwise_or(x.hour >= 20, x.hour <= 7),
    is_work_morning=lambda x: np.bitwise_and(x.hour >= 6, x.hour <= 10) & np.bitwise_not(x.is_weekend),
    is_summer=lambda x: x.month.between(6, 8),
    ctx_1=lambda x: x.percentage_docks_available.shift(1),
    ctx_2=lambda x: x.percentage_docks_available.shift(2),
    ctx_3=lambda x: x.percentage_docks_available.shift(3),
    ctx_4=lambda x: x.percentage_docks_available.shift(4),
    station_id_aux=lambda x: x.station_id.shift(4),
    altitude=lambda x: x.altitude.astype(int)
)

In [ ]:
df = df.query('not ctx_1.isnull()') \
    .query('not ctx_2.isnull()') \
    .query('not ctx_3.isnull()') \
    .query('not ctx_4.isnull()') \
    .query('station_id == station_id_aux') \
    .drop(columns=['station_id_aux']) \
    .query('not percentage_docks_available.isnull()')


In [ ]:
dfnans = df.isna().sum()
assert dfnans[dfnans > 0].empty

In [ ]:
df.head()

# CV

In [ ]:
from sklearn import model_selection, linear_model, ensemble, neighbors, preprocessing, metrics, pipeline, compose

In [ ]:
# CrossValidation

scoring = 'neg_root_mean_squared_error'


def get_cv_scores(model, X: pd.DataFrame, y: pd.Series, cv=4, verbose=0):
    if verbose > 0: display(f'{X.shape=}')
    if verbose > 0: display(f'{y.shape=}')

    return model_selection.cross_val_score(
        model, X, y,
        scoring=scoring,
        cv=model_selection.TimeSeriesSplit(cv),
        verbose=verbose,
    )

In [ ]:
sns.pairplot(
    data=df[[
        'lat',
        'lon',
        'altitude',
        'temperature_2m',
        'total_cloud_cover',
        'total_precipitation',
        'windspeed_10m',
        'day_of_week',
        'ctx_1',
        'ctx_2',
        'ctx_3',
        'ctx_4',
        'percentage_docks_available',
    ]].sample(frac=0.0001)
)
plt.show()

In [ ]:
plt.savefig('./fig.png')

In [ ]:
# Custom Regressor
from dataclasses import dataclass
from sklearn import linear_model
from typing import List, Dict


# create model per
@dataclass
class BicingRegression:
    grouping_column_name: str

    regressor_class: type = None
    verbose: int = 0

    groups_: List[int] = None
    regressors_: Dict[int, object] = None

    def __post_init__(self):
        self.regressors_ = self.regressors_ or {}
        self.regressor_class = self.regressor_class or linear_model.LinearRegression

    def fit(self, X, y, weights=None):
        if self.verbose >= 10:
            print(f'{type(X)=}')
            print(f'{type(y)=}')
        self.groups_ = X.loc[:, self.grouping_column_name].unique()

        for ii, group in enumerate(self.groups_):
            if self.verbose > 0:
                print(f'Training regressor {ii + 1}/{len(self.groups_)}')
            self.regressors_[group] = self.regressor_class()

            X_ = X.query(f'{self.grouping_column_name} == {group}')
            y_ = y.loc[X_.index]

            self.regressors_[group].fit(X_, y_)

        return self

    def predict(self, X):
        results = pd.DataFrame(
            index=X.index,
            columns=['prediction']
        )

        for group in self.groups_:
            X_ = X.query(f'{self.grouping_column_name} == {group}')
            if len(X_) == 0:
                continue
            y_pred = self.regressors_[group].predict(X_)
            results.loc[X_.index, 'prediction'] = y_pred

        return results['prediction']


In [ ]:
pipes = [
    {
        'model': pipeline.Pipeline(
            [
                ("transformer", compose.ColumnTransformer(
                    [
                        ('previous_hours', preprocessing.MinMaxScaler((0, 1)), [f"ctx_{ii + 1}" for ii in range(4)]),
                    ],
                    remainder="drop")
                 ),
                ("regressor", linear_model.LinearRegression())
            ]
        ),
        'weights': None,
    },
    {
        'model': pipeline.Pipeline(
            [
                ("transformer", compose.ColumnTransformer(
                    [
                        ('previous_hours', preprocessing.MinMaxScaler((0, 1)), [f"ctx_{ii + 1}" for ii in range(4)]),
                        ('std', preprocessing.StandardScaler(), ["altitude"]),
                    ],
                    remainder="drop", )
                 ),
                ("regressor", linear_model.LinearRegression())
            ]
        ),
        'weights': None,
    },
    {
        'model': pipeline.Pipeline(
            [
                ("transformer", compose.ColumnTransformer(
                    [
                        ('previous_hours', preprocessing.MinMaxScaler((0, 1)), [f"ctx_{ii + 1}" for ii in range(4)]),
                        ('std', preprocessing.StandardScaler(), ["temperature_2m"]),
                    ],
                    remainder="drop", )
                 ),
                ("regressor", linear_model.LinearRegression())
            ]
        ),
        'weights': None,
    },
    {
        'model': pipeline.Pipeline(
            [
                ("transformer", compose.ColumnTransformer(
                    [
                        ('previous_hours', preprocessing.MinMaxScaler((0, 1)), [f"ctx_{ii + 1}" for ii in range(4)]),
                        ('std', preprocessing.StandardScaler(), ["temperature_2m", "altitude"]),
                    ],
                    remainder="drop", )
                 ),
                ("regressor", linear_model.LinearRegression())
            ]
        ),
        'weights': None,
    },
    {
        'model': pipeline.Pipeline(
            [
                ("transformer", compose.ColumnTransformer(
                    [
                        ('previous_hours', preprocessing.MinMaxScaler((0, 1)), [f"ctx_{ii + 1}" for ii in range(4)]),
                        ('std', preprocessing.StandardScaler(), ["temperature_2m", "altitude"]),
                        ('coord', preprocessing.StandardScaler(), ["lat", "lon"]),
                        ('cat', preprocessing.OneHotEncoder(handle_unknown="ignore"), ["post_code"]),
                        ('clean', preprocessing.FunctionTransformer(feature_names_out='one-to-one'), ['station_id']),
                    ],
                    remainder="drop",
                    sparse_threshold=0,
                )
                 ),
                ("regressor", BicingRegression(
                    grouping_column_name='clean__station_id',
                    verbose=10
                ))
            ]
        ),
        'weights': None,
    },
    # {
    #     'model': pipeline.Pipeline(
    #         [
    #             ("transformer", compose.ColumnTransformer(
    #                 [
    #                     ('bool', preprocessing.OneHotEncoder(handle_unknown="ignore"),
    #                      ["is_weekend", "is_summer", "is_night"]),
    #                     ('previous_hours', preprocessing.MinMaxScaler((0, 1)), [f"ctx_{ii + 1}" for ii in range(4)]),
    #                     ('4bins', preprocessing.KBinsDiscretizer(n_bins=4, encode='onehot', strategy='uniform'),
    #                      ["hour", "month"]),
    #                     ('2bins', preprocessing.KBinsDiscretizer(n_bins=2, encode='onehot', strategy='uniform'),
    #                      ["temperature_2m"]),
    #                 ],
    #                 remainder="drop", )
    #              ),
    #             ("regressor", linear_model.LinearRegression())
    #         ]
    #     ),
    #     'weights': None,
    # },
]

Y_COLUMN = 'percentage_docks_available'

In [ ]:
df_ = df.sort_values('datetime')
SHOW_CV_STEPS = True
cv = model_selection.TimeSeriesSplit(2)

for ii, dd in enumerate(pipes[-1:]):
    pipe = dd['model']
    display(f'Running model with IDX: {ii}')
    display(pipe)

    # this is a sklearn 1.2.0 feature:
    pipe["transformer"].set_output('pandas')

    if SHOW_CV_STEPS:
        for ii, (train_idx, test_idx) in enumerate(cv.split(df_.drop(columns=[Y_COLUMN]))):
            df_train_ = df_.iloc[train_idx]
            df_test_ = df_.iloc[test_idx]

            pipe.fit(df_train_.drop(columns=[Y_COLUMN]), df_train_[Y_COLUMN])
            display(
                pd.Series(
                    data=pipe["regressor"].coef_,
                    index=pipe["transformer"].get_feature_names_out()
                )
            )
            y_test_pred = pipe.predict(df_test_.drop(columns=[Y_COLUMN]))
            sns.scatterplot(
                x=df_test_[Y_COLUMN],
                y=y_test_pred,
                alpha=0.2
            )
            plt.show()

    cv_scores = get_cv_scores(pipe, df_.drop(columns=[Y_COLUMN]), df_[Y_COLUMN], cv=3, verbose=0)
    display(f'{np.mean(cv_scores):0.4f}')

    display('-' * 80)



# final fit

In [ ]:
pipe.fit(df.drop(columns=[Y_COLUMN]), df[Y_COLUMN])

In [ ]:
# load X_val
X_val = pd.read_csv('./data/validation/X_validation.csv', index_col="index")
X_val = pd.merge(left=X_val, right=station_info[['station_id', 'lat', 'lon', 'altitude', 'post_code']],
                 on=['station_id'])
X_val = pd.merge(left=X_val, right=df_climate.drop(columns=['time']), on=['hour', 'day', 'month', 'year'])

X_val = X_val.assign(
    year=2023,
    date=lambda x: pd.to_datetime(dict(year=x.year, month=x.month, day=x.day)),
    day_of_week=lambda x: x.date.dt.day_of_week,
    is_weekend=lambda x: x.day_of_week >= 5,
    is_night=lambda x: np.bitwise_or(x.hour >= 20, x.hour <= 7),
    is_work_morning=lambda x: np.bitwise_and(x.hour >= 6, x.hour <= 10) & np.bitwise_not(x.is_weekend),
    is_summer=lambda x: x.month.between(6, 8),
).rename(
    columns={
        'ctx-1': 'ctx_1',
        'ctx-2': 'ctx_2',
        'ctx-3': 'ctx_3',
        'ctx-4': 'ctx_4',
    }
)

X_val.head()

In [133]:
n = 7
y_val = pd.DataFrame(
    pipe.predict(X_val),
    index=X_val.index,
    columns=[Y_COLUMN]
).assign(index=X_val.index)[['index', "percentage_docks_available"]]

display(y_val.head())

y_val.to_csv(f'./results/v2_{n}.csv', index=False)

array([ 3.32007294e-01, -1.32621617e-01,  3.93000430e-02,  8.28586837e-03,
       -1.42041137e+01, -8.62731006e-03, -1.27760547e+00,  0.00000000e+00,
       -2.55521093e+00,  0.00000000e+00])